# dependencies
connect to cpu if want to use behaviour cloning


In [ ]:
#!git clone http://github.com/HumanCompatibleAI/imitation
#!pip install -e "/content/imitation[dev]"

In [ ]:
!pip install gymnasium[mujoco]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 17.3 MB/s eta 0:00:00


In [ ]:
!pip install imitation
!pip install shimmy
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823

# environment and expert

In [ ]:
import gymnasium as gym
import imitation
import seals

In [ ]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.algorithms import bc
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
import sb3_contrib
from sb3_contrib import TRPO
from imitation.data import rollout
import torch
from torch.optim.adam import Adam
SEED = 42

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def make_env_expert_transitions(n_trajs):
    # Create the environment, load expert policy, and sample trajectories
    # from the expert policy and environment.
    global SEED
    env = make_vec_env(
        "Reacher-v4",
        rng=np.random.default_rng(SEED),
        n_envs=1,
        post_wrappers=[
            lambda env, _: RolloutInfoWrapper(env)
        ],  # needed for computing rollouts later
        parallel=True,
    )
    expert = load_policy(
        "ppo",
        path = "/content/ppo_Reacher-v4.zip", #-self-trained Reacher expert
        venv=env,
    )
    rollouts = rollout.rollout(
        expert,
        env,
        rollout.make_sample_until(min_timesteps=1000, min_episodes = n_trajs),
        rng=np.random.default_rng(SEED), #The random state to use for sampling trajectories
    )
    transitions = rollout.flatten_trajectories(rollouts)

    return env, expert, transitions


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from typing import Callable
from stable_baselines3 import PPO
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func

In [ ]:
def make_BC_trainer(env, expert, transitions):
    global SEED
# 可以调policy network的架构
# Custom actor (pi) and value function (vf) networks
# of two layers of size 32 each with Relu activation function
#policy_kwargs = dict(activation_fn=th.nn.ReLU,
#                     net_arch=[dict(pi=[32, 32], vf=[32, 32])])
    policy_network = MlpPolicy(observation_space = env.observation_space,
                            action_space = env.action_space,
                            lr_schedule = linear_schedule(0.0003034620018780926))

    bc_trainer = bc.BC(
        observation_space=env.observation_space,
        action_space=env.action_space,
        policy = policy_network, # default optimizer is adam
        batch_size = 32,
        optimizer_kwargs = {"lr": 0.0003034620018780926},
        demonstrations=transitions,
        rng=np.random.default_rng(SEED),
    )
    return bc_trainer

# BC - 1 trajectory

In [ ]:
env_1, expert_1, transitions_1 = make_env_expert_transitions(1)

In [ ]:
bc_trainer_1 = make_BC_trainer(env_1, expert_1, transitions_1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
expert_reward, _ = evaluate_policy(
    expert_1, env_1, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: -4.46325095 +/- 2.4631429161322833


In [ ]:
env_1.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    bc_trainer_1.policy, env_1, 20, return_episode_rewards=True
)

In [ ]:
env_1.seed(SEED)
bc_trainer_1.train(n_epochs = 60, log_interval = 500)

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00284 |
|    entropy        | 2.84     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 151      |
|    loss           | 1.84     |
|    neglogp        | 1.84     |
|    prob_true_act  | 0.158    |
|    samples_so_far | 32       |
--------------------------------


16batch [00:00, 154.77batch/s]
57batch [00:00, 87.83batch/s] 
79batch [00:00, 117.87batch/s]
118batch [00:00, 150.82batch/s]
138batch [00:00, 163.41batch/s]
175batch [00:01, 158.03batch/s]
212batch [00:01, 167.55batch/s]
248batch [00:01, 168.54batch/s]
266batch [00:01, 166.84batch/s]
303batch [00:01, 175.31batch/s]
339batch [00:02, 167.94batch/s]
359batch [00:02, 176.33batch/s]
397batch [00:02, 180.71batch/s]
434batch [00:02, 166.25batch/s]
451batch [00:02, 166.97batch/s]
488batch [00:03, 175.67batch/s]
Epoch 15 of 60                 

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 500      |
|    ent_loss       | -0.00253 |
|    entropy        | 2.53     |
|    epoch          | 16       |
|    l2_loss        | 0        |
|    l2_norm        | 156      |
|    loss           | 1.53     |
|    neglogp        | 1.54     |
|    prob_true_act  | 0.215    |
|    samples_so_far | 16032    |
--------------------------------


524batch [00:03, 170.29batch/s]
544batch [00:03, 178.16batch/s]
582batch [00:03, 179.42batch/s]
618batch [00:03, 174.59batch/s]
636batch [00:03, 173.36batch/s]
674batch [00:04, 180.06batch/s]
713batch [00:04, 187.39batch/s]
733batch [00:04, 188.67batch/s]
771batch [00:04, 187.97batch/s]
790batch [00:04, 181.68batch/s]
828batch [00:04, 179.88batch/s]
865batch [00:05, 168.86batch/s]
883batch [00:05, 169.97batch/s]
928batch [00:05, 195.73batch/s]
948batch [00:05, 190.85batch/s]
987batch [00:05, 183.36batch/s]
Epoch 31 of 60                 

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 1000     |
|    ent_loss       | -0.00223 |
|    entropy        | 2.23     |
|    epoch          | 32       |
|    l2_loss        | 0        |
|    l2_norm        | 159      |
|    loss           | 1.23     |
|    neglogp        | 1.24     |
|    prob_true_act  | 0.291    |
|    samples_so_far | 32032    |
--------------------------------


1006batch [00:05, 174.72batch/s]
1049batch [00:06, 195.63batch/s]
1070batch [00:06, 199.69batch/s]
1114batch [00:06, 208.08batch/s]
1135batch [00:06, 207.94batch/s]
1157batch [00:06, 211.22batch/s]
1203batch [00:06, 218.71batch/s]
1225batch [00:06, 212.65batch/s]
1271batch [00:07, 218.18batch/s]
1293batch [00:07, 210.53batch/s]
1315batch [00:07, 204.81batch/s]
1357batch [00:07, 198.00batch/s]
1377batch [00:07, 189.82batch/s]
1417batch [00:07, 185.50batch/s]
1456batch [00:08, 188.48batch/s]
1475batch [00:08, 182.35batch/s]
1494batch [00:08, 183.35batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 1500     |
|    ent_loss       | -0.00193 |
|    entropy        | 1.93     |
|    epoch          | 48       |
|    l2_loss        | 0        |
|    l2_norm        | 161      |
|    loss           | 0.929    |
|    neglogp        | 0.931    |
|    prob_true_act  | 0.394    |
|    samples_so_far | 48032    |
--------------------------------


1513batch [00:08, 182.70batch/s]
1532batch [00:08, 183.64batch/s]
1573batch [00:08, 190.12batch/s]
1593batch [00:08, 187.94batch/s]
1635batch [00:09, 186.76batch/s]
1655batch [00:09, 190.23batch/s]
1697batch [00:09, 195.84batch/s]
1719batch [00:09, 202.11batch/s]
1761batch [00:09, 181.96batch/s]
1781batch [00:09, 186.38batch/s]
1821batch [00:10, 176.34batch/s]
1841batch [00:10, 182.15batch/s]
1860batch [00:10, 180.82batch/s]


In [ ]:
env_1.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    bc_trainer_1.policy, env_1, 20, return_episode_rewards=True
)

In [ ]:
print(
    "learner rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

learner rewards before training: -12.066415799999998 +/- 4.1685616352329085


In [ ]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: -5.957656900000001 +/- 2.63302779656932


In [ ]:
import torch
torch.save(bc_trainer_1.policy, "BC_MlpPolicy_Reacher_1.pth.tar")

# BC - 4 trajectory

In [ ]:
env_4, expert_4, transitions_4 = make_env_expert_transitions(4)

In [ ]:
bc_trainer_4 = make_BC_trainer(env_4, expert_4, transitions_4)

In [ ]:
expert_reward, _ = evaluate_policy(
    expert_4, env_4, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: -4.46325095 +/- 2.4631429161322833


In [ ]:
env_4.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    bc_trainer_4.policy, env_4, 20, return_episode_rewards=True
)

In [ ]:
env_4.seed(SEED)
bc_trainer_4.train(n_epochs = 80, log_interval = 500)

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00284 |
|    entropy        | 2.84     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 151      |
|    loss           | 1.84     |
|    neglogp        | 1.84     |
|    prob_true_act  | 0.158    |
|    samples_so_far | 32       |
--------------------------------


22batch [00:00, 211.40batch/s]
44batch [00:00, 207.08batch/s]
89batch [00:00, 217.52batch/s]
112batch [00:00, 219.71batch/s]
134batch [00:00, 218.30batch/s]
179batch [00:00, 219.59batch/s]
201batch [00:00, 213.81batch/s]
248batch [00:01, 222.30batch/s]
271batch [00:01, 220.87batch/s]
294batch [00:01, 222.43batch/s]
341batch [00:01, 223.14batch/s]
364batch [00:01, 221.42batch/s]
387batch [00:01, 222.33batch/s]
434batch [00:01, 226.90batch/s]
457batch [00:02, 222.15batch/s]
480batch [00:02, 216.14batch/s]
Epoch 15 of 80                 

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 500      |
|    ent_loss       | -0.00253 |
|    entropy        | 2.53     |
|    epoch          | 16       |
|    l2_loss        | 0        |
|    l2_norm        | 156      |
|    loss           | 1.53     |
|    neglogp        | 1.54     |
|    prob_true_act  | 0.215    |
|    samples_so_far | 16032    |
--------------------------------


526batch [00:02, 219.64batch/s]
548batch [00:02, 214.51batch/s]
571batch [00:02, 217.79batch/s]
619batch [00:02, 227.28batch/s]
642batch [00:02, 225.10batch/s]
665batch [00:03, 218.35batch/s]
689batch [00:03, 222.06batch/s]
738batch [00:03, 227.85batch/s]
761batch [00:03, 225.08batch/s]
784batch [00:03, 219.60batch/s]
832batch [00:03, 223.05batch/s]
855batch [00:03, 222.89batch/s]
878batch [00:03, 221.62batch/s]
925batch [00:04, 224.95batch/s]
949batch [00:04, 226.75batch/s]
972batch [00:04, 226.37batch/s]
995batch [00:04, 223.60batch/s]

...

1018batch [00:04, 225.24batch/s]
1041batch [00:04, 224.69batch/s]
1064batch [00:04, 222.31batch/s]
1113batch [00:05, 230.03batch/s]
1137batch [00:05, 223.50batch/s]
1160batch [00:05, 223.70batch/s]
1206batch [00:05, 220.68batch/s]
1229batch [00:05, 221.77batch/s]
1252batch [00:05, 222.73batch/s]
1300batch [00:05, 228.47batch/s]
1323batch [00:05, 222.86batch/s]
1346batch [00:06, 215.67batch/s]
1393batch [00:06, 223.58batch/s]
1416batch [00:06, 213.74batch/s]
1438batch [00:06, 212.03batch/s]
1485batch [00:06, 221.49batch/s]
Epoch 47 of 80                  

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 1500     |
|    ent_loss       | -0.00193 |
|    entropy        | 1.93     |
|    epoch          | 48       |
|    l2_loss        | 0        |
|    l2_norm        | 162      |
|    loss           | 0.929    |
|    neglogp        | 0.931    |
|    prob_true_act  | 0.394    |
|    samples_so_far | 48032    |
--------------------------------


1508batch [00:06, 214.79batch/s]
1530batch [00:06, 213.24batch/s]
1575batch [00:07, 216.62batch/s]
1597batch [00:07, 216.12batch/s]
1642batch [00:07, 215.56batch/s]
1664batch [00:07, 211.40batch/s]
1686batch [00:07, 207.94batch/s]
1733batch [00:07, 219.40batch/s]
1755batch [00:07, 215.56batch/s]
1777batch [00:08, 215.64batch/s]
1822batch [00:08, 218.52batch/s]
1844batch [00:08, 216.24batch/s]
1888batch [00:08, 212.63batch/s]
1910batch [00:08, 203.70batch/s]
1931batch [00:08, 205.43batch/s]
1977batch [00:09, 214.60batch/s]
1999batch [00:09, 210.13batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 2000     |
|    ent_loss       | -0.00162 |
|    entropy        | 1.62     |
|    epoch          | 64       |
|    l2_loss        | 0        |
|    l2_norm        | 164      |
|    loss           | 0.629    |
|    neglogp        | 0.631    |
|    prob_true_act  | 0.532    |
|    samples_so_far | 64032    |
--------------------------------



2044batch [00:09, 212.08batch/s]
2066batch [00:09, 214.16batch/s]
2089batch [00:09, 216.49batch/s]
2135batch [00:09, 221.20batch/s]
2158batch [00:09, 218.04batch/s]
2180batch [00:09, 214.38batch/s]
2225batch [00:10, 197.94batch/s]
2246batch [00:10, 193.00batch/s]
2286batch [00:10, 191.50batch/s]
2306batch [00:10, 184.68batch/s]
2346batch [00:10, 189.75batch/s]
2386batch [00:11, 191.19batch/s]
2406batch [00:11, 191.42batch/s]
2445batch [00:11, 189.04batch/s]
2464batch [00:11, 184.30batch/s]
2480batch [00:11, 213.90batch/s]


In [ ]:
env_4.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    bc_trainer_4.policy, env_4, 20, return_episode_rewards=True
)

In [ ]:
print(
    "learner rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

learner rewards before training: -12.021541249999999 +/- 4.020863542059987


In [ ]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: -5.6513271000000005 +/- 2.4352558276847405


In [ ]:
import torch
torch.save(bc_trainer_4.policy, "BC_MlpPolicy_Reacher_4.pth.tar")

# BC - 10 trajectory

In [ ]:
env_10, expert_10, transitions_10 = make_env_expert_transitions(10)

In [ ]:
bc_trainer_10 = make_BC_trainer(env_10, expert_10, transitions_10)

In [ ]:
expert_reward, _ = evaluate_policy(
    expert_10, env_10, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: -4.46325095 +/- 2.4631429161322833


In [ ]:
env_10.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    bc_trainer_10.policy, env_10, 20, return_episode_rewards=True
)

In [ ]:
env_10.seed(SEED)
bc_trainer_10.train(n_epochs = 70, log_interval = 500)

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00284 |
|    entropy        | 2.84     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 151      |
|    loss           | 1.84     |
|    neglogp        | 1.84     |
|    prob_true_act  | 0.158    |
|    samples_so_far | 32       |
--------------------------------


19batch [00:00, 187.53batch/s]
55batch [00:00, 157.99batch/s]
89batch [00:00, 161.86batch/s]
106batch [00:00, 148.12batch/s]
145batch [00:00, 170.87batch/s]
183batch [00:01, 176.98batch/s]
202batch [00:01, 180.17batch/s]
241batch [00:01, 187.33batch/s]
279batch [00:01, 180.19batch/s]
298batch [00:01, 168.26batch/s]
336batch [00:01, 177.38batch/s]
354batch [00:02, 171.69batch/s]
398batch [00:02, 196.02batch/s]
421batch [00:02, 204.54batch/s]
444batch [00:02, 210.03batch/s]
491batch [00:02, 220.46batch/s]
Epoch 15 of 70                 

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 500      |
|    ent_loss       | -0.00253 |
|    entropy        | 2.53     |
|    epoch          | 16       |
|    l2_loss        | 0        |
|    l2_norm        | 156      |
|    loss           | 1.54     |
|    neglogp        | 1.54     |
|    prob_true_act  | 0.215    |
|    samples_so_far | 16032    |
--------------------------------


514batch [00:02, 217.02batch/s]
536batch [00:02, 217.33batch/s]
582batch [00:03, 215.97batch/s]
605batch [00:03, 218.97batch/s]
628batch [00:03, 219.79batch/s]
675batch [00:03, 222.77batch/s]
698batch [00:03, 219.59batch/s]
744batch [00:03, 223.13batch/s]
767batch [00:03, 219.95batch/s]
790batch [00:04, 208.59batch/s]
834batch [00:04, 212.70batch/s]
856batch [00:04, 214.36batch/s]
878batch [00:04, 206.46batch/s]
924batch [00:04, 216.42batch/s]
946batch [00:04, 217.38batch/s]
969batch [00:04, 219.26batch/s]
993batch [00:04, 221.29batch/s]

...

1016batch [00:05, 215.65batch/s]
1038batch [00:05, 210.86batch/s]
1082batch [00:05, 200.36batch/s]
1103batch [00:05, 193.42batch/s]
1147batch [00:05, 204.26batch/s]
1170batch [00:05, 209.75batch/s]
1192batch [00:05, 212.22batch/s]
1237batch [00:06, 212.74batch/s]
1260batch [00:06, 215.98batch/s]
1282batch [00:06, 213.14batch/s]
1326batch [00:06, 204.15batch/s]
1347batch [00:06, 204.59batch/s]
1393batch [00:06, 216.45batch/s]
1415batch [00:07, 216.08batch/s]
1437batch [00:07, 215.63batch/s]
1482batch [00:07, 219.25batch/s]
Epoch 47 of 70                  

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 1500     |
|    ent_loss       | -0.00193 |
|    entropy        | 1.93     |
|    epoch          | 48       |
|    l2_loss        | 0        |
|    l2_norm        | 161      |
|    loss           | 0.931    |
|    neglogp        | 0.933    |
|    prob_true_act  | 0.393    |
|    samples_so_far | 48032    |
--------------------------------


1504batch [00:07, 217.68batch/s]
1549batch [00:07, 217.24batch/s]
1572batch [00:07, 220.27batch/s]
1595batch [00:07, 217.16batch/s]
1640batch [00:08, 216.62batch/s]
1662batch [00:08, 214.79batch/s]
1684batch [00:08, 215.10batch/s]
1730batch [00:08, 220.93batch/s]
1753batch [00:08, 221.25batch/s]
1777batch [00:08, 224.43batch/s]
1823batch [00:08, 220.07batch/s]
1846batch [00:09, 216.41batch/s]
1868batch [00:09, 216.23batch/s]
1916batch [00:09, 221.60batch/s]
1939batch [00:09, 219.96batch/s]
1962batch [00:09, 218.47batch/s]
1985batch [00:09, 216.86batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 2000     |
|    ent_loss       | -0.00162 |
|    entropy        | 1.62     |
|    epoch          | 64       |
|    l2_loss        | 0        |
|    l2_norm        | 164      |
|    loss           | 0.629    |
|    neglogp        | 0.63     |
|    prob_true_act  | 0.532    |
|    samples_so_far | 64032    |
--------------------------------


2007batch [00:09, 202.06batch/s]
2028batch [00:09, 202.80batch/s]
2075batch [00:10, 215.21batch/s]
2097batch [00:10, 209.81batch/s]
2119batch [00:10, 212.20batch/s]
2165batch [00:10, 219.82batch/s]
2170batch [00:10, 206.09batch/s]


In [ ]:
env_10.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    bc_trainer_10.policy, env_10, 20, return_episode_rewards=True
)

In [ ]:
print(
    "learner rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

learner rewards before training: -11.9900719 +/- 4.051810178667849


In [ ]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: -5.8411083 +/- 2.229874162802491


In [ ]:
import torch
torch.save(bc_trainer_10.policy, "BC_MlpPolicy_Reacher_10.pth.tar")